In [1]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [2]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
input_prefix = 'data/'
#input_prefix = 'https://github.com/hjort/ai-labs/raw/master/jupyter/sales-russia/data/'

bzfile = '.bz2' # ''

In [4]:
#dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
df_train = pd.read_csv(\
    input_prefix + 'sales_train.csv' + bzfile,
    #nrows=300, # somente primeiras linhas!
    #parse_dates=['date'], date_parser=dateparse,
    dtype={'date_block_num': np.int8, 'shop_id': np.int8, 'item_id': np.int16, \
           'item_price': np.float32, 'item_cnt_day': np.int32},
    )
print('shape:', df_train.shape)
df_train.head()

shape: (2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1
1,03.01.2013,0,25,2552,899.00,1
2,05.01.2013,0,25,2552,899.00,-1
3,06.01.2013,0,25,2554,1709.05,1
4,15.01.2013,0,25,2555,1099.00,1


In [5]:
%time df_train['item_price'] = (np.ceil(df_train['item_price'])).astype(np.int32)

CPU times: user 13 ms, sys: 207 µs, total: 13.2 ms
Wall time: 12.5 ms


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int8
shop_id           int8
item_id           int16
item_price        int32
item_cnt_day      int32
dtypes: int16(1), int32(2), int8(2), object(1)
memory usage: 56.0+ MB


In [7]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.9358e+06,2.9358e+06,2.9358e+06,2.9358e+06,2.9358e+06
mean,1.4570e+01,3.3002e+01,1.0197e+04,8.9089e+02,1.2426e+00
std,9.4230e+00,1.6227e+01,6.3243e+03,1.7298e+03,2.6188e+00
min,0.0000e+00,0.0000e+00,0.0000e+00,-1.0000e+00,-2.2000e+01
25%,7.0000e+00,2.2000e+01,4.4760e+03,2.4900e+02,1.0000e+00
50%,1.4000e+01,3.1000e+01,9.3430e+03,3.9900e+02,1.0000e+00
75%,2.3000e+01,4.7000e+01,1.5684e+04,9.9900e+02,1.0000e+00
max,3.3000e+01,5.9000e+01,2.2169e+04,3.0798e+05,2.1690e+03


In [8]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999,1
1,03.01.2013,0,25,2552,899,1
2,05.01.2013,0,25,2552,899,-1
3,06.01.2013,0,25,2554,1710,1
4,15.01.2013,0,25,2555,1099,1


In [9]:
df_train2 = df_train.query('item_cnt_day > 0 and item_price > 0')
df_train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2928492 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int8
shop_id           int8
item_id           int16
item_price        int32
item_cnt_day      int32
dtypes: int16(1), int32(2), int8(2), object(1)
memory usage: 78.2+ MB


In [10]:
item_price_monthly_mean = np.ceil(
        df_train2.groupby(
            ['shop_id', 'item_id', 'date_block_num']
        )['item_price'].mean()
    ).astype(np.int32)
item_price_monthly_mean

shop_id  item_id  date_block_num
0        30       1                  265
         31       1                  434
         32       0                  221
                  1                  221
         33       0                  347
                  1                  347
         35       0                  247
                  1                  247
         36       1                  357
         40       1                  127
         42       1                  127
         43       0                  221
         49       1                  127
         51       0                  129
                  1                  127
         57       1                  167
         59       1                  110
         61       0                  195
         75       0                   76
         85       1                  190
         88       0                   76
         95       0                  193
         96       0                   70
         97       1     

In [11]:
item_cnt_monthly_sum = df_train2.groupby(
        ['shop_id', 'item_id', 'date_block_num']
    )['item_cnt_day'].sum()
item_cnt_monthly_sum

shop_id  item_id  date_block_num
0        30       1                 31
         31       1                 11
         32       0                  6
                  1                 10
         33       0                  3
                  1                  3
         35       0                  1
                  1                 14
         36       1                  1
         40       1                  1
         42       1                  1
         43       0                  1
         49       1                  2
         51       0                  2
                  1                  3
         57       1                  1
         59       1                  1
         61       0                  1
         75       0                  1
         85       1                  1
         88       0                  1
         95       0                  1
         96       0                  1
         97       1                  1
         98       0            

In [12]:
type(item_price_monthly_mean)

pandas.core.series.Series

In [13]:
type(item_cnt_monthly_sum)

pandas.core.series.Series

In [14]:
item_monthly = pd.concat([item_price_monthly_mean, item_cnt_monthly_sum], axis=1)

#item_monthly = pd.merge(
#    item_price_monthly_mean, item_cnt_monthly_sum, 
#    how='inner', left_index=True, right_index=True)

item_monthly.head()

item_price  item_cnt_day
shop_id item_id date_block_num                          
0       30      1                      265            31
        31      1                      434            11
        32      0                      221             6
                1                      221            10
        33      0                      347             3

In [15]:
del(item_price_monthly_mean)
del(item_cnt_monthly_sum)

In [16]:
item_monthly.rename(columns={
        'item_price': 'item_price_mean', 'item_cnt_day': 'item_cnt_sum'
    }, inplace=True)
item_monthly.head()

item_price_mean  item_cnt_sum
shop_id item_id date_block_num                               
0       30      1                           265            31
        31      1                           434            11
        32      0                           221             6
                1                           221            10
        33      0                           347             3

In [17]:
item_monthly.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1608226 entries, (0, 30, 1) to (59, 22167, 17)
Data columns (total 2 columns):
item_price_mean    1608226 non-null int32
item_cnt_sum       1608226 non-null int32
dtypes: int32(2)
memory usage: 18.6 MB


In [18]:
item_monthly.describe().T

,count,mean,std,min,25%,50%,75%,max
item_price_mean,1608226.0,790.1960,1549.3138,1.0,199.0,399.0,895.0,307980.0
item_cnt_sum,1608226.0,2.2732,8.6532,1.0,1.0,1.0,2.0,2253.0


In [19]:
item_monthly.index.names

FrozenList(['shop_id', 'item_id', 'date_block_num'])

In [20]:
item_monthly.to_csv('sales_monthly.csv')

In [21]:
!head sales_monthly.csv

shop_id,item_id,date_block_num,item_price_mean,item_cnt_sum
0,30,1,265,31
0,31,1,434,11
0,32,0,221,6
0,32,1,221,10
0,33,0,347,3
0,33,1,347,3
0,35,0,247,1
0,35,1,247,14
0,36,1,357,1


In [22]:
!rm -f sales_monthly.csv.bz2 && bzip2 -9 sales_monthly.csv